In [1]:
import os
import pandas as pd
import numpy as np
import itertools as it

In [14]:
sub = 'pilot-54'
ses = 'ses-02'
battery = 'fbirn'
task = 'breath-hold'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
# main_dir = '/home/sshankar/Documents/Data/'
datadir = os.path.join(main_dir, sub)
csv_file = 'extracted_breath-hold_sub-54_ses-02_2112021-1159.csv'
tsv_file = 'breath-hold_pilot-54_ses_02.tsv'

In [15]:
file_df = pd.read_csv(os.path.join(datadir, csv_file))

In [16]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['sscanstart1.OffsetTime'])])['sscanstart1.OffsetTime']
run_start = run_start.values[0]

In [17]:
run_start

57041.0

In [18]:
# Event onsets are in 'Breathe.OnsetTime', 'GetReady.OnsetTime' and 'HoldBreath.OnsetTime'
breathe_onsets = (file_df.loc[~np.isnan(file_df['Breathe.OnsetTime'])])['Breathe.OnsetTime']
getready_onsets = (file_df.loc[~np.isnan(file_df['GetReady.OnsetTime'])])['GetReady.OnsetTime']
holdbreath_onsets = (file_df.loc[~np.isnan(file_df['HoldBreath.OnsetTime'])])['HoldBreath.OnsetTime']

# How many trials were presented
n_trials = len(breathe_onsets)

# Trial offsets are in 'CatImage.OffsetTime'
breathe_offsets = (file_df.loc[~np.isnan(file_df['Breathe.OffsetTime'])])['Breathe.OffsetTime']
getready_offsets = (file_df.loc[~np.isnan(file_df['GetReady.OffsetTime'])])['GetReady.OffsetTime']
holdbreath_offsets = (file_df.loc[~np.isnan(file_df['HoldBreath.OffsetTime'])])['HoldBreath.OffsetTime']

trial_onsets = np.sort(np.hstack((breathe_onsets, getready_onsets, holdbreath_onsets)))

# This task doesn't have any responses, duration is offset-onset times
breathe_durations = breathe_offsets - breathe_onsets
getready_durations = getready_offsets - getready_onsets
holdbreath_durations = holdbreath_offsets - holdbreath_onsets

durations = []
for b in zip(breathe_durations, getready_durations, holdbreath_durations):
    durations.extend(list(it.chain(b)))
    
# Finally, trial_types are in 'Running'
trial_types = ['breathe', 'get_ready', 'hold_breath'] * n_trials

In [73]:
trial_types

['breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath',
 'breathe',
 'get_ready',
 'hold_breath']

In [74]:
durations

[17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0,
 17900.0,
 1900.0,
 15900.0]

In [19]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(trial_onsets-run_start)/1000, 'duration':np.array(durations)/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)